## Waterhackweek cyberseminar
# Workflows for gridded climate datasets
## Diana Gergel and Bart Nijssen
### University of Washington
### February 14, 2019

**Abstract:** Climate change, forecasting, satellite datasets, large model ensembles ... Large gridded datasets are everywhere in hydrology and earth science. While accessing and analyzing these datasets required some serious programming skills not so long ago, a number of toolkits are now available that let you easily access, ingest, analyze and display gridded climate datasets. In this webinar we’ll discuss one of the most common file formats used in our field for large data sets, the Network Common Data Format (NetCDF), and step through a Jupyter notebook to showcase python packages, such as xarray and cartopy, that can be used to examine them. No prior experience required, although we will build on some of the skills you have acquired in earlier webinars in the series.

## GitHub repository

The GitHub repository for this presentation can be found at: https://github.com/bartnijssen/gridded_data

The README.md file in this repository explains how to configure the environment you need to run the sample notebooks.

## Outline

* `gridded_data_1_background.ipynb`


* `gridded_data_2_xarray_and_dask.ipynb`


* `gridded_data_3_cartopy_and_seaborn.ipynb` (this notebook)

    * Creating a heatmap with `seaborn`
    * Creating a global map with features using `cartopy`


* `gridded_data_4_rasterio.ipynb`

Load some packages that we'll be using: 

In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np 
import os 
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

__To start with, let's load the DataSet that we left off with in the last notebook:__

In [ ]:
tbot = xr.open_mfdataset(os.path.join('data', 
                                      'b.e11*'), concat_dim='time')

 __We're going to start off by looking at using `seaborn` (https://seaborn.pydata.org/), a data visualization library based on matplotlib. It integrates well with `pandas` and can also be used in conjunction with `cartopy`, which we'll talk about in a bit.__ 

__Let's first look at an example from the Seaborn gallery (https://seaborn.pydata.org/examples/index.html), which is a great place to go if you're interested in how you can use `seaborn` in your research.__

In [ ]:
sns.set(style="white")
plt.figure(figsize=(8, 4))

# Load the example mpg dataset
mpg = sns.load_dataset("mpg")

# Plot miles per gallon against horsepower with other semantics
sns.relplot(x="horsepower", y="mpg", hue="origin", size="weight",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=4, data=mpg)

__The disadvantage of `seaborn` is that to use this code, our DataFrame needs to be structured pretty similar to this one:__

In [ ]:
mpg[:5]

__Let's take a look at another example, and this time let's get our data to be in the right format:__

__We're taking this example heatmap from the Seaborn gallery:__

In [ ]:
sns.set()

# Load the example flights dataset and conver to long-form
flights_long = sns.load_dataset("flights")
flights = flights_long.pivot("month", "year", "passengers")

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(flights, annot=True, fmt="d", linewidths=.5, ax=ax)

__We'll start off with selecting one spatial point from our DataSet, this is located in the southern part of South America__: 

__And before we do that we'll subset our DataSet on the time axis:__

In [ ]:
tbot_time_slice = tbot['TBOT'].sel(time=slice('2070-01-01', '2099-12-31'))

In [ ]:
tbot_pt = tbot_time_slice[:, 45, 233]

__Now we've lost our spatial dimension and just have a time series:__

In [ ]:
tbot_pt

__We need our DataFrame to be formatted similarly to the example one:__

In [ ]:
flights_long[:5]

__So let's create our DataFrame with the subsetted `TBOT`:__

In [ ]:
tbot_df = pd.DataFrame()
tbot_df['year'] = tbot_time_slice['time.year'].values
tbot_df['month'] = tbot_time_slice['time.month'].values
tbot_df['tbot'] = (tbot_pt - 273.15).values

__And let's see what it looks like:__

In [ ]:
tbot_df[:5]

__Now let's make our heatmap:__

In [ ]:
sns.set()

f, ax = plt.subplots(figsize=(8,4))
tbot_pivot = tbot_df.pivot("month", "year", "tbot")
sns.heatmap(tbot_pivot, 
            linewidths=.5, 
            ax=ax, 
            cbar_kws={'label': 'Temperature ($^\circ$C)'}, 
            cmap='viridis')

__Now we'll shift to a quick intro to `cartopy` (https://scitools.org.uk/cartopy/docs/latest/), a package designed for geospatial data processing to produce maps and other analyses.__

__We'll start by visualizing the mean for the last 30 years of the 21st century:__

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

(tbot_time_slice.mean('time') - 273.15).plot(ax=ax, transform=ccrs.PlateCarree(), 
                                             subplot_kws={'projection': ccrs.Robinson()}, 
                                             cbar_kwargs={'label': 'Temperature ($^\circ$C)'})
ax.add_feature(cfeature.OCEAN)

__We can add some country borders too:__

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

(tbot_time_slice.mean('time') - 273.15).plot(ax=ax, transform=ccrs.PlateCarree(), 
                                             subplot_kws={'projection': ccrs.Robinson()}, 
                                             cbar_kwargs={'label': 'Temperature ($^\circ$C)'})
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

__And then also lakes and rivers:__

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

(tbot_time_slice.mean('time') - 273.15).plot(ax=ax, transform=ccrs.PlateCarree(), 
                                             subplot_kws={'projection': ccrs.Robinson()}, 
                                             cbar_kwargs={'label': 'Temperature ($^\circ$C)'})
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES, alpha=0.8)
ax.add_feature(cfeature.RIVERS, alpha=0.8)

__There are lots of other projections available too: https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html#cartopy-projections__

__We can also define our own classes of projections, which I've done for the Arctic for my research.__

__If you'd like to learn more about `cartopy`, the `cartopy` site is a great place to explore: https://scitools.org.uk/cartopy/docs/latest/index.html__